In [1]:
from autogen_ext.models.ollama import OllamaChatCompletionClient
from autogen_core.models import UserMessage, ModelFamily

ollama_client = OllamaChatCompletionClient(
    model="qwen2.5:7b-instruct",
    model_info = {
        'vision': False,
        'function_calling': True,
        'json_output': True,
        'family': ModelFamily.GPT_4O
    },
    options={
        'num_ctx': 6000,}
)

In [3]:
from email.mime import base
import os
from autogen_agentchat.teams import MagenticOneGroupChat
from autogen_agentchat.ui import Console
from autogen_agentchat.conditions import TextMentionTermination
from autogen_ext.agents.file_surfer import FileSurfer  # 导入正确的FileSurferAgent类

# 创建 FileSurfer 代理
file_surfer = FileSurfer(
    name="FileSurferAgent",
    model_client=ollama_client,
    base_path=os.path.expanduser("~/dev/shallow_edu/course/docs"),
)

# 定义终止条件
termination_condition = TextMentionTermination("APPROVE")

# 创建 MagenticOneGroupChat 团队
team = MagenticOneGroupChat([file_surfer], model_client=ollama_client, termination_condition=termination_condition)

# 定义任务，例如读取本地文件
task = "读取本地文件 c1.txt 的内容并总结。"

# 运行任务
await Console(team.run_stream(task=task))


---------- TextMessage (user) ----------
读取本地文件 c1.txt 的内容并总结。
---------- TextMessage (MagenticOneOrchestrator) ----------

We are working to address the following user request:

读取本地文件 c1.txt 的内容并总结。


To answer this request we have assembled the following team:

FileSurferAgent: An agent that can handle local files.


Here is an initial fact sheet to consider:

1. GIVEN OR VERIFIED FACTS
   - The request asks to read the content of a local file named "c1.txt".
   
2. FACTS TO LOOK UP
   - There are no specific facts or figures given in the request that need to be looked up.

3. FACTS TO DERIVE
   - No logical deductions, simulations, or computations are required based on the provided information.

4. EDUCATED GUESSES
   - None of the information provided allows for any educated guesses.


Here is the plan to follow as best as possible:

- FileSurferAgent: Read the content of the local file "c1.txt".
- Summarize the content read from the file "c1.txt".

---------- TextMessage (Magenti

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='读取本地文件 c1.txt 的内容并总结。', type='TextMessage'), TextMessage(source='MagenticOneOrchestrator', models_usage=None, metadata={}, content='\nWe are working to address the following user request:\n\n读取本地文件 c1.txt 的内容并总结。\n\n\nTo answer this request we have assembled the following team:\n\nFileSurferAgent: An agent that can handle local files.\n\n\nHere is an initial fact sheet to consider:\n\n1. GIVEN OR VERIFIED FACTS\n   - The request asks to read the content of a local file named "c1.txt".\n   \n2. FACTS TO LOOK UP\n   - There are no specific facts or figures given in the request that need to be looked up.\n\n3. FACTS TO DERIVE\n   - No logical deductions, simulations, or computations are required based on the provided information.\n\n4. EDUCATED GUESSES\n   - None of the information provided allows for any educated guesses.\n\n\nHere is the plan to follow as best as possible:\n\n- FileSurferAgent: Read

In [ ]:
response = await ollama_client.create([UserMessage(content="中国首都是?", source="user")])
print(response.content)

In [ ]:
import asyncio

from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.base import TaskResult
from autogen_agentchat.conditions import ExternalTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create an OpenAI model client.
model_client = ollama_client

# Create the primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=model_client,
    system_message="You are a helpful AI assistant.",
)

# Create the critic agent.
critic_agent = AssistantAgent(
    "critic",
    model_client=model_client,
    system_message="Provide constructive feedback. Respond with 'APPROVE' to when your feedbacks are addressed.",
)

# Define a termination condition that stops the task if the critic approves.
text_termination = TextMentionTermination("APPROVE")

# Create a team with the primary and critic agents.
team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=text_termination)


In [ ]:
await team.reset()  # Reset the team for a new task.
await Console(team.run_stream(task="写一首五言绝句，关于夏天的热"))  # Stream the messages to the console.



In [ ]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
import os
import asyncio



# 创建文件读取 Agent
file_reader_agent = AssistantAgent(
    name="FileReaderAgent",
    model_client=ollama_client,
    description="An agent that can read and process local text files.",
    system_message="""You are an agent capable of reading local text files. 
    When asked to read a file, use the provided file path and return its contents.
    If the file doesn't exist or there's an error, report it clearly."""
)

# 创建用户代理
user_proxy = UserProxyAgent(
    name="UserProxy",
    #model_client=ollama_client,
    code_execution_config={
        "work_dir": "docs",
        "use_docker": False  # 设置为 False 以在本地运行
    }
)

# 定义读取文件的函数
def read_local_file(file_path: str) -> str:
    try:
        if not os.path.exists(file_path):
            return f"Error: File '{file_path}' does not exist."
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
        return content
    except Exception as e:
        return f"Error reading file '{file_path}': {str(e)}"

# 注册文件读取功能到 agent
file_reader_agent.register_function(
    function_map={
        "read_local_file": read_local_file
    }
)

# 在 Notebook 中运行异步代码
async def main():
    file_path = "test.txt"
    result = await user_proxy.initiate_chat(
        recipient=file_reader_agent,
        message=f"Please read the content of the file: {file_path}"
    )
    return result.content

# 执行异步任务
result = await main()
print(result)